In [35]:
import stim
import numpy as np
from sklearn.model_selection import train_test_split

distance=3
num_ancilla_qubits=8
rounds=5

surface_code_circuit = stim.Circuit.generated(
    "surface_code:rotated_memory_x",
    rounds=5,
    distance=3,
    after_clifford_depolarization=0.01,
    after_reset_flip_probability=0.01,
    before_measure_flip_probability=0.01,
    before_round_data_depolarization=0.01)

num_shots=100*64
# Compile the sampler
sampler = surface_code_circuit.compile_detector_sampler()
# Sample shots, with observables
detection_events, observable_flips = sampler.sample(num_shots, separate_observables=True)


detection_events = detection_events.astype(int)
detection_strings = [''.join(map(str, row)) for row in detection_events] #compress the detection events in a tensor
detection_events_numeric = [[int(value) for value in row] for row in detection_events] # Convert string elements to integers (or floats if needed)
detection_array = np.array(detection_events_numeric) # Convert detection_events to a numpy array
print(detection_array[0])

detection_array1 = detection_array.reshape(num_shots, rounds, num_ancilla_qubits) #first dim is the number of shots, second dim round number, third dim is the Ancilla 
print(detection_array1[0]) 

observable_flips = observable_flips.astype(int).flatten().tolist()

[1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0]
[[1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchviz import make_dot


class FullyConnectedNN(nn.Module):
    def __init__(self, input_size, layers_sizes, hidden_size):
        super(FullyConnectedNN, self).__init__()
        
        # Define the input layer
        self.input_layer = nn.Linear(input_size, layers_sizes[0])
        
        # Define hidden layers
        self.hidden_layers = nn.ModuleList()
        for i in range(len(layers_sizes) - 1):
            self.hidden_layers.append(nn.Linear(layers_sizes[i], layers_sizes[i + 1]))
        
        # Define output layer
        self.output_layer = nn.Linear(layers_sizes[-1], hidden_size)

        # Define activation function (e.g., ReLU)
        self.activation = nn.ReLU()

    def forward(self, x):
        # Pass through input layer
        #torch.transpose(x.squeeze(0), 1,0)

        x = self.activation(self.input_layer(x))
        
        # Pass through hidden layers
        for layer in self.hidden_layers:
            x = self.activation(layer(x))
        
        # Pass through output layer
        x = self.output_layer(x)
        
        return x

In [ ]:
class LatticeRNNCell(nn.Module):
    def __init__(self, input_size, hidden_size,layers_sizes,batch_size):
        super(LatticeRNNCell, self).__init__()
        self.hidden=hidden_size
        self.batch_size=batch_size
        self.fc_input = nn.Linear(input_size, input_size)
        #self.fc_hidden_double = nn.Linear(hidden_size*3, hidden_size)
        self.multi_layer_fc = FullyConnectedNN(hidden_size*3, layers_sizes, hidden_size)
        self.multi_layer_fc1 = FullyConnectedNN(hidden_size*3, layers_sizes, hidden_size)
        self.lstm_cell = nn.LSTMCell(input_size, hidden_size)

    def forward(self, x, hidden_left, hidden_up, hidden_pre, hidden_pre1):
        # if hidden_left is not None and hidden_bottom is not None:
        #     hidden = (hidden_left + hidden_bottom) / 2  # Average of left and bottom
        # elif hidden_left is not None:
        #     hidden = hidden_left
        # elif hidden_bottom is not None:
        #     hidden = hidden_bottom
        # else:
        #     hidden = torch.zeros(x.size(0), self.hidden_size).to(x.device)  # Initial hidden state
        
        #input fc net
        #input=self.fc_input(x.type(torch.FloatTensor))

        """
        # Combine the hidden states from left and bottom
        if hidden_left is not None and hidden_up is not None:
            combined_hidden=torch.cat((hidden_left,hidden_up),1)
            hidden=self.fc_hidden_double(combined_hidden.squeeze(0)).unsqueeze(0)

        elif hidden_left is not None:
            hidden=self.fc_hidden_single(hidden_left.squeeze(0)).unsqueeze(0)

        elif hidden_up is not None:
            hidden=self.fc_hidden_single(hidden_up.squeeze(0)).unsqueeze(0)
        else:
            hidden=torch.zeros(1,self.hidden, dtype=torch.float)
        """
        
        hidden_pre=torch.Tensor(hidden_pre)
        hidden_pre1=torch.Tensor(hidden_pre1)

        # Combine the hidden states from left and bottom
        if hidden_left[0] is not None and hidden_up[0] is not None:
            combined_hidden=torch.cat((hidden_left[0],hidden_up[0], hidden_pre),1)
        elif hidden_left[0] is not None:
            hidden_init_up=torch.zeros(self.batch_size,self.hidden, dtype=torch.float)
            combined_hidden=torch.cat((hidden_left[0],hidden_init_up, hidden_pre),1)
        elif hidden_up[0] is not None:
            hidden_init_left=torch.zeros(self.batch_size,self.hidden, dtype=torch.float)
            combined_hidden=torch.cat((hidden_init_left,hidden_up[0], hidden_pre),1)
        else:
            hidden_init_left=torch.zeros(self.batch_size,self.hidden, dtype=torch.float)
            hidden_init_up=torch.zeros(self.batch_size,self.hidden, dtype=torch.float)
            combined_hidden=torch.cat((hidden_init_left,hidden_init_up, hidden_pre),1)



        if hidden_left[1] is not None and hidden_up[1] is not None:
            combined_hidden1=torch.cat((hidden_left[1],hidden_up[1], hidden_pre1),1)
        elif hidden_left[1] is not None:
            hidden_up=torch.zeros(self.batch_size,self.hidden, dtype=torch.float)
            combined_hidden1=torch.cat((hidden_left[1],hidden_up, hidden_pre1),1)
        elif hidden_up[1] is not None:
            hidden_left=torch.zeros(self.batch_size,self.hidden, dtype=torch.float)
            combined_hidden1=torch.cat((hidden_left,hidden_up[1], hidden_pre1),1)
        else:
            hidden_left=torch.zeros(self.batch_size,self.hidden, dtype=torch.float)
            hidden_up=torch.zeros(self.batch_size,self.hidden, dtype=torch.float)
            combined_hidden1=torch.cat((hidden_left,hidden_up, hidden_pre1),1)


        hidden=self.multi_layer_fc(combined_hidden)
        hidden1=self.multi_layer_fc1(combined_hidden1)

        x=x.squeeze(1).float()

        # Update hidden state using current input and combined hidden state
        hidden,hidden1 = self.lstm_cell(x, (hidden,hidden1))
        
        return hidden,hidden1

class LatticeRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, grid_height, grid_width,layers_sizes, batch_size):
        super(LatticeRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.grid_height = grid_height
        self.grid_width = grid_width 
        self.lstm_cells = nn.ModuleList([LatticeRNNCell(input_size, hidden_size, layers_sizes, batch_size) for _ in range(grid_height * grid_width)])
        self.fc_hidden = nn.Linear(hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()  # For binary output
    
    def forward(self, x, hidden_ext, hidden_ext1, grid):
        
        # Reshape the input to match the grid size
        batch_size, seq_len, _ = x.size()
        x = x.reshape(batch_size,self.grid_height, self.grid_width)

        for i in range(self.grid_height):
            for j in range(self.grid_width):

                input_bit = x[:,i,j].unsqueeze(1).unsqueeze(1)  # Get the input for the current cell
                (hidden_pre, hidden_pre1)=grid[i][j]

                if i==0 and j==0:
                    hidden_left=(hidden_ext,hidden_ext1)

                if j > 0 :
                    hidden_left = grid[i][j - 1]
                else:
                    hidden_left = (None,None)

                if i > 0:
                    hidden_up = grid[i-1][j]
                else:
                    hidden_up = (None,None)

                

                # Get the index for the current RNN cell
                cell_index = i * self.grid_width + j
                hidden, hidden1 = self.lstm_cells[cell_index](input_bit, hidden_left, hidden_up, hidden_pre, hidden_pre1)

                # Store the hidden state in the grid
                grid[i][j] = (hidden,hidden1)

        # The output that matters is the hidden state from the top-right corner (i.e., grid[grid_size-1][grid_size-1])
        final_hidden,  final_hidden1 = grid[-1][-1]

        # Pass the hidden state through the fully connected layer and sigmoid for binary output
        #hidden= self.fc_hidden(bottom_right_hidden)
        output = self.fc_out(final_hidden)
        output = self.sigmoid(output)

        return output,  final_hidden, final_hidden1, grid
    


# RNN model
class BlockRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, grid_height, grid_width, rounds,layers_sizes,batch_size):
        super(BlockRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.fc_in = nn.Linear(input_size, input_size)
        self.rnn_block = LatticeRNN(input_size, hidden_size, output_size, grid_height, grid_width, layers_sizes,batch_size)
        self.fc_out = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()  # For binary output
    
    def forward(self, x, rounds):
        #initialize the external hidden states
        hidden_ext = torch.zeros(self.batch_size,self.hidden_size) # (1,hidden_size) hidden state of the previous round 
        hidden_ext1 = torch.zeros(self.batch_size,self.hidden_size)

        #Initialize the previous hidden state for each element of the grid
        grid = [[(hidden_ext,hidden_ext1) for _ in range(grid_width)] for _ in range(grid_height)]

        for round in range (rounds):
            input_block = x[:,round,:].unsqueeze(2)  # (1, 8) syndromes
            out, hidden_ext, hidden_ext1,grid = self.rnn_block(input_block, hidden_ext, hidden_ext1, grid)

        #I already use fc, sigmoid in the LatticeRNN
        #out = self.fc_out(out)  # Use the last time-step's output, needed for changing the dimension of the output compared of input
        #out = self.sigmoid(out)  # I need a Binary output
        return out, hidden_ext[0]
    
    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size)

In [38]:
def train_rnn(model, X_train, y_train, criterion, optimizer, num_epochs, batch_size,rounds):
    
    model.train()  # Set the model to training mode
    # If batch_size is None, set it to the full size of the dataset (no mini-batching)

    num_samples = len(X_train[:,0,0])
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        # # Shuffle the dataset at the beginning of each epoch
        # permutation = torch.randperm(num_samples)
        # X_train = X_train[permutation]
        # y_train = y_train[permutation]

        # Mini-batch training loop
        for i in range(0, num_samples, batch_size):
            # Create mini-batches
            batch_x = torch.from_numpy(X_train[i:i + batch_size])
            batch_y = torch.Tensor(y_train[i:i + batch_size])

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            #hidden = model.init_hidden(1)
            output, hidden= model(batch_x, rounds)

            # Compute loss
            loss = criterion(output.squeeze(1), batch_y)

            # Backward pass (compute gradients)
            loss.backward()

            # Optimize (update weights)
            optimizer.step()

            # Accumulate the loss for logging purposes
            running_loss += loss.item()

        # Print average loss after each epoch
        avg_loss = running_loss / (num_samples // batch_size)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    print("Training finished.")

In [39]:
def binary_array_to_tensor(binary_array):
    # Check if the input is a NumPy array, if not, convert it
    if isinstance(binary_array, np.ndarray):
        tensor = torch.from_numpy(binary_array).float()  # Convert NumPy array to float32 tensor
    else:
        # If not a NumPy array, convert it as before
        tensor = torch.tensor([[int(bit) for bit in binary_array]], dtype=torch.float32)
    return tensor.unsqueeze(0)  # Add batch dimension (batch_size = 1)


def test(model, test_sequences, targets,batch_size):
    model.eval()  # Set the model to evaluation mode (disable dropout, etc.)
    correct = 0

    hidden = None
    num_samples = len(test_sequences[:,0,0])
    
    
    with torch.no_grad():  # Disable gradient computation for testing
        for i in range(0, num_samples, batch_size):
            
            output=np.zeros(batch_size)
            batch_x = torch.from_numpy(test_sequences[i:i + batch_size])
            target = targets[i:i + batch_size]
            rounds = len(batch_x[0,:,0])
            
            # Initialize hidden state
            #if hidden is None:
                # Initialize hidden state for the first sequence (or batch)
            #    hidden = model.init_hidden(batch_size=1)  # batch_size might vary depending on your use case
            #else:
                #hidden=hidden.detach() #you detach if you want to avoid that the gradient is propagated through the hidden states to avoid long training time and memory usage
            
            # Forward pass for prediction
            output, hidden = model(batch_x, rounds)
            prediction = torch.round(output)  # Convert probability to binary (0 or 1)
            
            for j in range(0,batch_size):
                if prediction[j] == target[j]:
                    correct += 1
    
    accuracy = correct / len(test_sequences)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

In [40]:
# Example usage
input_size = 1  # Each RNN cell takes 1 bit as input
hidden_size = 8 # Hidden size of each RNN cell
output_size = 1  # Binary output
grid_height = 4  # 4x2 grid of RNN cells
grid_width= 2
batch_size=64
layers_sizes=[hidden_size*3,hidden_size*2,hidden_size ]

model = LatticeRNN(input_size, hidden_size, output_size, grid_height, grid_width, layers_sizes,batch_size)

# Input is a batch of binary sequences (batch_size, seq_len, input_size), reshaped to match grid size
seq_len = grid_height * grid_width  # Number of bits equal to the grid size squared
x = torch.randn(batch_size, seq_len, input_size)  # Random input, replace with binary input
hidden_ext = torch.randn(batch_size, hidden_size)  # Random input, replace with binary input
hidden_ext1 = torch.randn(batch_size, hidden_size)  # Random input, replace with binary input
grid = [[(hidden_ext,hidden_ext1) for _ in range(grid_width)] for _ in range(grid_height)]
print(grid[0][0])

output, hidden, hidden1,grid = model(x,hidden_ext,hidden_ext1,grid)
print(hidden)


(tensor([[-0.3828, -0.2308,  1.7810, -0.8717, -1.4502,  0.6449, -0.7657, -0.8595],
        [ 0.4465,  0.2631, -0.3668,  0.8292, -1.7754,  0.5092,  0.4778, -1.2875],
        [ 1.0268,  1.1072,  1.5153, -0.9435,  0.3933, -1.8176,  0.2264,  0.2579],
        [-0.6780, -0.0839, -0.3555,  0.6451, -0.4284, -0.8158,  0.0876,  0.2736],
        [ 1.5853,  0.2665, -1.1151,  1.3386, -1.7707, -2.8244,  1.5124,  2.1033],
        [ 0.6984, -0.1893,  1.4000, -0.2501, -0.5194,  0.8266, -1.4237,  0.9600],
        [ 0.8913,  0.4565,  1.4860, -0.3937, -1.1715, -0.4154,  0.4492,  0.9243],
        [ 1.2519,  0.6777, -0.8730,  1.0986, -1.3260,  1.3426,  1.3883, -2.5794],
        [-0.3898,  0.3868, -0.9496,  0.1856, -0.0998, -0.7518, -0.2682,  0.1330],
        [ 2.5882,  1.1027, -0.3544, -1.6151,  2.1255, -0.5815, -1.3279, -1.7784],
        [ 0.5494, -0.9577, -0.3117,  0.2608, -1.7650,  1.7253,  0.1102,  1.0625],
        [-0.5680, -0.3036, -1.7354,  0.4713, -0.5229,  1.1842,  0.0547, -0.1647],
        [ 1.232

In [41]:
# Hyperparameters
input_size = 1  # Each Lattice RNN cell takes 1 bit as input
hidden_size = 64 # Hidden size of each RNN cell
output_size = 1  # Binary output (e.g., 0 or 1)
grid_height = 4  # Number of rows in the grid
grid_width = 2   # Number of columns in the grid
learning_rate = 0.0005
num_epochs = 10
batch_size = 64
layers_sizes=[hidden_size*3,hidden_size*2,hidden_size ]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create a model instance
model = BlockRNN(input_size, hidden_size, output_size, grid_height, grid_width, rounds,layers_sizes,batch_size).to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary cross-entropy loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

test_size=0.2
test_dataset_size=num_shots*test_size
X_train, X_test, y_train, y_test = train_test_split(detection_array1, observable_flips, test_size=0.2, random_state=42, shuffle=False)

# Training the model
train_rnn(model, X_train, y_train, criterion, optimizer, num_epochs,batch_size,rounds)

test(model, X_test, y_test,batch_size)

Epoch [1/10], Loss: 0.6069
Epoch [2/10], Loss: 0.5614
Epoch [3/10], Loss: 0.5610
Epoch [4/10], Loss: 0.5607
Epoch [5/10], Loss: 0.5543
Epoch [6/10], Loss: 0.5228
Epoch [7/10], Loss: 0.5230
Epoch [8/10], Loss: 0.5211
Epoch [9/10], Loss: 0.5159
Epoch [10/10], Loss: 0.5059
Training finished.
Test Accuracy: 74.22%
